<a href="https://colab.research.google.com/github/luciotravassos/colab/blob/main/agrupamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **APRENDIZAGEM NÃO SUPERVISIONADA: AGRUPAMENTO**

Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para agrupar clientes de um shopping. 

Os dados foram extraídos do site do Kaggle:

https://www.kaggle.com/shwetabh123/mall-customers

In [3]:
import numpy as np
import pandas as pd

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI/Treinamento/Mall_Customers.csv',
                    sep=',', encoding='iso-8859-1')

In [7]:
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


**ATRIBUTOS:** 

CustomerID: identificação do cliente.

Genre: Gênero.

Age: idade.

Annual Income (k$):	Rendimento anual.

Spending Score (1 a 100): Pontuação de gastos.

**IDADE**

In [8]:
import plotly.express as px

In [9]:
hist =  px.histogram (df,  x = "Age", nbins=60) 
hist.update_layout(width=600,height=400,title_text='Distribuição das idades') 
hist.show()

**GÊNERO**

In [10]:
df['Genre'].value_counts()

Female    112
Male       88
Name: Genre, dtype: int64

In [11]:
import seaborn as sns

In [ ]:
sns.countplot(x='Genre', data=df);

In [13]:
df.shape

(200, 5)

## **EXPLORAÇÃO E TRATAMENTO DOS DADOS**

**Alterando nomes das colunas**

In [14]:
df.head(2)

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81


In [15]:
df.rename(columns={'CustomerID': 'identificacao'}, inplace=True)

In [16]:
df.rename(columns={'Genre': 'genero'}, inplace=True)
df.rename(columns={'Age': 'idade'}, inplace=True)

In [17]:
df.rename(columns={'Annual Income (k$)': 'rendimento','Spending Score (1-100)':'pontuacao'}, inplace=True) 

In [18]:
df.head(3)

,identificacao,genero,idade,rendimento,pontuacao
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6


**Valores Missing (NAN)**

In [19]:
# RELAÇÃO DA QUANTIDADE
df.isnull().sum()

identificacao    0
genero           0
idade            0
rendimento       0
pontuacao        0
dtype: int64

**Análise dos tipos de atributos.**

In [20]:
# object: strings
# int64: inteiros
# float64: reais
# complex: complexos
df.dtypes

identificacao     int64
genero           object
idade             int64
rendimento        int64
pontuacao         int64
dtype: object

**Dados Estatísticos**

In [21]:
df.describe()

,identificacao,idade,rendimento,pontuacao
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


**Análise dos outliers**

In [22]:
import plotly.express as px

In [23]:
boxplot = px.box(df, y="rendimento")
boxplot.show()

In [24]:
boxplot = px.box(df, y="idade")
boxplot.show()

In [25]:
boxplot = px.box(df, y="pontuacao")
boxplot.show()

## **PRÉ-PROCESSAMENTO**

In [26]:
df.head(2)

,identificacao,genero,idade,rendimento,pontuacao
0,1,Male,19,15,39
1,2,Male,21,15,81


###**Excluir identificação do cliente**

In [27]:
df2 = df.drop(labels = 'identificacao', axis = 1)

In [28]:
df2.head(2)

,genero,idade,rendimento,pontuacao
0,Male,19,15,39
1,Male,21,15,81


###**Transformar variáveis categóricas em numéricas**

In [29]:
df2['genero'].replace({'Female': 0, 'Male': 1}, inplace=True)

In [30]:
df2.head(3)

,genero,idade,rendimento,pontuacao
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6


###**Escalonamento**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
escala = StandardScaler()
df2_esc = escala.fit_transform(df2)

In [ ]:
df2_esc

## **K-MEANS COM DOIS ATRIBUTOS**

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html 

In [ ]:
df2

In [ ]:
dados = df2.iloc[:, [2,3]].values

In [ ]:
dados

**Elbow Method**

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 5, max_iter = 300)
    kmeans.fit(dados)
    # inertia: Método para gerar o wcss
    wcss.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(range(1, 11), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS');

**Agrupamento**

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 5, max_iter = 300)


In [ ]:
kmeans1 = kmeans.fit(dados)

In [ ]:
# Coordenadas dos Centróides
centroides = kmeans1.cluster_centers_
centroides

In [ ]:
# Classificação dos dados
classificacao = kmeans1.labels_
classificacao

In [ ]:
dados[3,:]

In [ ]:
classificacao[3]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
graf1 = px.scatter(x = dados[:,0], y = dados[:,1], color=classificacao)
graf2 = px.scatter(x = centroides[:,0], y = centroides[:,1], size = [15, 15, 15, 15])
graf3 = go.Figure(data = graf1.data + graf2.data)
graf3.update_layout(width=800,height=500,title_text='Agrupamento K-Means')
graf3.update_xaxes(title = 'Rendimento anual (K$)')
graf3.update_yaxes(title = 'Pontuação')
graf3.show()

In [ ]:
agrupamento = pd.DataFrame(classificacao, columns = ['Grupo'])
agrupamento

In [ ]:
df3 = pd.concat([df, agrupamento],axis=1)
df3

In [ ]:
df3.iloc[150 , :]

In [ ]:
df3.loc[df3.identificacao == 92] 

In [ ]:
grupo0 = df3.loc[df3.Grupo == 0]
grupo0 

In [ ]:
grupo0.shape

In [ ]:
grupo1 = df3.loc[df3.Grupo == 1]
grupo1

In [ ]:
grupo1.shape

## **K-MEANS COM TODOS ATRIBUTOS**

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Elbow Method**

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,15):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 5, max_iter = 300)
    kmeans.fit(df2_esc)
# Método inertia para gerar o wcss
    wcss.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(range(1, 15), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS');

**Agrupamento**

In [ ]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++', random_state = 5, max_iter = 300)


In [ ]:
kmeans2 = kmeans.fit(df2_esc)

In [ ]:
# Coordenadas dos Centróides
centroides = kmeans2.cluster_centers_
centroides

In [ ]:
# Inversão do escalonamento
escala.inverse_transform(kmeans2.cluster_centers_)

In [ ]:
# Classificação dos dados
classificacao2 = kmeans2.labels_
classificacao2

In [ ]:
classificacao2[199]

In [ ]:
agrupamento2 = pd.DataFrame(classificacao2, columns = ['Grupo'])
agrupamento2

In [ ]:
df4 = pd.concat([df, agrupamento2],axis=1)
df4

In [ ]:
df4.iloc[125 , :]

In [ ]:
df4.loc[df4.identificacao == 76] 

In [ ]:
grupo0 = df4.loc[df4.Grupo == 0]
grupo0 

In [ ]:
grupo0.shape

## **K-MEANS COM PCA**

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Componentes Principais**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
df2_pca = pca.fit_transform(df2_esc)

In [ ]:
df2_pca

In [ ]:
# razão das variáveis explicativas
pca.explained_variance_ratio_

In [ ]:
# Soma das variáveis explicativas
pca.explained_variance_ratio_.sum()

**Elbow Method**

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 5, max_iter = 300)
    kmeans.fit(df2_pca)
# Método inertia para gerar o wcss
    wcss.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(range(1, 11), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS');

**Agrupamento**

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 5, max_iter = 300)


In [ ]:
kmeans3 = kmeans.fit(df2_pca)

In [ ]:
# Coordenadas dos Centróides
centroides = kmeans3.cluster_centers_
centroides

In [ ]:
# Classificação dos dados
classificacao3 = kmeans3.labels_
classificacao3

In [ ]:
classificacao3[2]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
graf1 = px.scatter(x = df2_pca[:,0], y = df2_pca[:,1], color=classificacao3)
graf2 = px.scatter(x = centroides[:,0], y = centroides[:,1], size = [15, 15, 15, 15])
graf3 = go.Figure(data = graf1.data + graf2.data)
graf3.update_layout(width=800,height=500,title_text='Agrupamento K-Means com PCA')
graf3.update_xaxes(title = 'Componente 1')
graf3.update_yaxes(title = 'Componente 2')
graf3.show()

In [ ]:
agrupamento3 = pd.DataFrame(classificacao3, columns = ['Grupo'])
agrupamento3

In [ ]:
df5 = pd.concat([df, agrupamento3],axis=1)
df5

In [ ]:
df5.iloc[125 , :]

In [ ]:
df5.loc[df5.identificacao == 76] 

In [ ]:
grupo0 = df5.loc[df5.Grupo == 0]
grupo0 

In [ ]:
grupo0.shape

## **ALGORITMO HIERÁRQUICO COM PCA**

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html 

In [ ]:
df2.head()

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Componentes Principais**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
df2_pca = pca.fit_transform(df2_esc)

In [ ]:
df2_pca

**Dendograma**

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
dendrograma = dendrogram(linkage(df2_pca, method = 'complete'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
hier = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage = 'complete')
classificacao4 = hier.fit_predict(df2_pca)

In [ ]:
classificacao4

In [ ]:
import plotly.express as px

In [ ]:
graf = px.scatter(x = df2_pca[:,0], y = df2_pca[:,1], color=classificacao4)
graf.update_layout(width=800,height=500,title_text='Agrupamento Hierárquico')
graf.show()

**Agrupamento**

In [ ]:
agrupamento4 = pd.DataFrame(classificacao4, columns = ['Grupo'])
agrupamento4

In [ ]:
df6 = pd.concat([df, agrupamento4],axis=1)
df6

In [ ]:
df6.loc[df6.identificacao == 22] 

In [ ]:
grupo2 = df6.loc[df6.Grupo == 2]
grupo2 

## **ALGORITMO HIERÁRQUICO: todos atributos**






In [ ]:
df2.head()

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Dendograma**

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
dendrograma = dendrogram(linkage(df2_esc, method = 'average'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
hier = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage = 'average')
classificacao5 = hier.fit_predict(df2_esc)

In [ ]:
classificacao5

**Agrupamento**

In [ ]:
agrupamento5 = pd.DataFrame(classificacao5, columns = ['Grupo'])
agrupamento5

In [ ]:
df7 = pd.concat([df, agrupamento5],axis=1)
df7

In [ ]:
df7.loc[df7.identificacao == 131]

In [ ]:
grupo4 = df7.loc[df7.Grupo == 6]
grupo4.head(2)

In [ ]:
grupo4.shape

## **ALGORITMO DBSCAN COM PCA**

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html 

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Componentes Principais**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
df2_pca = pca.fit_transform(df2_esc)

In [ ]:
df2_pca

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps = 0.32, min_samples=4)
dbscan.fit(df2_pca)

In [ ]:
classificacao6 = dbscan.labels_
classificacao6

In [ ]:
import plotly.express as px

In [ ]:
graf = px.scatter(x = df2_pca[:,0], y = df2_pca[:,1], color=classificacao6)
graf.update_layout(width=800,height=500,title_text='Agrupamento DBSCAN')
graf.show()

**Agrupamento**

In [ ]:
agrupamento6 = pd.DataFrame(classificacao6, columns = ['Grupo'])
agrupamento6

In [ ]:
df8 = pd.concat([df, agrupamento6],axis=1)
df8

In [ ]:
df8.loc[df8.identificacao == 5] 

In [ ]:
ruido = df8.loc[df8.Grupo == -1]
ruido 

In [ ]:
ruido.shape

## **ALGORITMO DBSCAN: todos atributos**

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps = 0.73, min_samples=4)
dbscan.fit(df2_esc)

In [ ]:
classificacao7 = dbscan.labels_
classificacao7

**Agrupamento**

In [ ]:
agrupamento7 = pd.DataFrame(classificacao7, columns = ['Grupo'])
agrupamento7

In [ ]:
df9 = pd.concat([df, agrupamento7],axis=1)
df9

In [ ]:
df9.loc[df9.identificacao == 127] 

In [ ]:
ruido = df9.loc[df9.Grupo == -1]
ruido 

In [ ]:
ruido.shape

In [ ]:
grupo = df9.loc[df9.Grupo == 1]
grupo.shape

## **ALGORITMO MEANSHIFT COM PCA**

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html

In [ ]:
df2_esc

In [ ]:
df2_esc.shape

**Componentes Principais**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
df2_pca = pca.fit_transform(df2_esc)

In [ ]:
df2_pca

**MeanShift**

In [ ]:
from sklearn.cluster import MeanShift

In [ ]:
mean = MeanShift(bandwidth=1, cluster_all=True)

In [ ]:
mean.fit(df2_pca)

In [ ]:
classificacao8 = mean.labels_
classificacao8

In [ ]:
import plotly.express as px

In [ ]:
graf = px.scatter(x = df2_pca[:,0], y = df2_pca[:,1], color=classificacao8)
graf.update_layout(width=800,height=500,title_text='Agrupamento MeanShift')
graf.show()

**Agrupamento**

In [ ]:
agrupamento8 = pd.DataFrame(classificacao8, columns = ['Grupo'])
agrupamento8

In [ ]:
df9 = pd.concat([df, agrupamento8],axis=1)
df9

In [ ]:
df9.loc[df9.identificacao == 19] 

In [ ]:
ruido = df9.loc[df9.Grupo == -1]
ruido 

In [ ]:
grupo = df9.loc[df9.Grupo == 3]
grupo.shape

## **ALGORITMO MEANSHIFT: todos atributos**

In [ ]:
df2_esc

In [ ]:
df2

In [ ]:
df2_esc.shape

In [ ]:
from sklearn.cluster import MeanShift

In [ ]:
mean = MeanShift(bandwidth=1.5, cluster_all=True)

In [ ]:
mean.fit(df2_esc)

In [ ]:
classificacao9 = mean.labels_
classificacao9

**Agrupamento**

In [ ]:
agrupamento9 = pd.DataFrame(classificacao9, columns = ['Grupo'])
agrupamento9

In [ ]:
df10 = pd.concat([df, agrupamento9],axis=1)
df10

In [ ]:
df10.loc[df10.identificacao == 45] 

In [ ]:
grupo = df10.loc[df10.Grupo == 4]
grupo.shape

## **K-PROTOTYPES**

Algoritmo para agrupamento de dados mistos (categóricos e numéricos). Documentação: https://pypi.org/project/kmodes/

---



In [ ]:
df.head()

In [ ]:
dados= df[['genero','idade','rendimento', 'pontuacao']]
dados

In [ ]:
pip install kmodes

In [ ]:
from kmodes.kprototypes import KPrototypes

In [ ]:
kproto = KPrototypes(n_clusters= 4)

In [ ]:
kproto1 = kproto.fit_predict(dados, categorical=[0])

In [ ]:
kproto1

In [ ]:
kproto1[4]

In [ ]:
agrupamento = pd.DataFrame(kproto1, columns = ['grupo'])
agrupamento

In [ ]:
df_final = pd.concat([df, agrupamento],axis=1)

In [ ]:
df_final.head()

In [ ]:
df_final.iloc[132 , :]

In [ ]:
grupo = df_final.loc[df_final.grupo ==3]
grupo.head()

In [ ]:
grupo.shape

## **K-MODES**

Algoritmo para agrupamento de dados categóricos. Documentação: https://pypi.org/project/kmodes/

https://www.kaggle.com/datasets/yukeshmarudhasalam/bankmarketing 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cursos_Udemy/Machine_Learning_com_Python/bankmarketing.csv',
                    sep=',', encoding='iso-8859-1')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

**Variáveis**

**age** : idade (numérica)

**job** : tipo de trabalho (categorias: 'admin.','blue-collar','entrepreneur(empreendedor)','housemaid','management (gestão)','retired (aposentado)','self-employed (autônomo)','services','student','technician','unemployed (desempregado)','unknown')

**marital** : estado civil (categorias: 'divorced','married','single','unknown'; obs.: 'divorced' significa divorciado ou viúvo (a))

**education**: nível escolar (categorias: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

**default**: inadimplência (categorias: 'no','yes','unknown')

**housing**: crédito de habitação (categorias: 'no','yes','unknown')

**loan**: empréstimo pessoal (categorias: 'no','yes','unknown')

**contact**: contato (categorias: 'cellular','telephone')

**month**: último mês de contato (categorias: 'jan', 'feb', 'mar', 'april', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')

**day_of_week**: último dia de contato (categorical: 'mon','tue','wed','thu','fri')

**poutcome:** resultado do marketing anterior (categorical: 'failure','nonexistent','success')

In [ ]:
df2 = df[['age','job', 'marital', 'education', 'default', 'housing', 'loan','contact','month','day_of_week','poutcome']]
df2

In [ ]:
df3 = df2.copy()

In [ ]:
df3['faixa_idade'] = pd.cut(df3['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                              labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])

In [ ]:
df3  = df3.drop('age',axis = 1)

In [ ]:
df3.head(30)

In [ ]:
df3.dtypes

In [ ]:
df3.isnull().sum()

### **Pré-processamento**

**Transformação para codificação numérica**

In [ ]:
from sklearn import preprocessing
df4 = df3.apply(preprocessing.LabelEncoder().fit_transform)
df4.head()

### **Instalação**

In [ ]:
pip install kmodes

In [ ]:
from kmodes.kmodes import KModes

In [ ]:
?KModes

### **NÚMERO DE CLUSTERS "IDEAL"**

In [ ]:
custo = []
for num_clusters in list(range(1,10)):
    kmode = KModes(n_clusters=num_clusters, n_init = 2, verbose=1)
    kmode.fit_predict(df4)
    custo.append(kmode.cost_)

In [ ]:
import matplotlib.pyplot as plt
y = np.array([i for i in range(1,10,1)])
plt.plot(y,custo);

In [ ]:
kmd = KModes(n_clusters=4, verbose=1)
kmd1 = kmd.fit_predict(df4)

In [ ]:
kmd1

In [ ]:
kmd1[40000]

In [ ]:
agrupamento = pd.DataFrame(kmd1, columns = ['grupo'])
agrupamento

In [ ]:
df5 = pd.concat([df2, agrupamento],axis=1)

In [ ]:
df5.head()

In [ ]:
df5.iloc[33284 , :]

In [ ]:
grupo = df5.loc[df5.grupo == 3]
grupo.head()

In [ ]:
grupo.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.subplots(figsize = (15,5))
sns.countplot(x=df5['job'], order=df5['job'].value_counts().index, hue=df5['grupo'])
plt.show()

## **DESAFIO 5**

DESENVOLVER E SELECIONAR O MELHOR ALGORITMO DE MACHINE LEARNING DE AGRUPAMENTO PARA O DATASET DO LINK A SEGUIR:

https://www.kaggle.com/harrywang/wine-dataset-for-clustering 